## --- Day 1: Sonar Sweep --- (Part 1)

You're minding your own business on a ship at sea when the overboard alarm goes off! You rush to see if you can help. Apparently, one of the Elves tripped and accidentally sent the sleigh keys flying into the ocean!

Before you know it, you're inside a submarine the Elves keep ready for situations like this. It's covered in Christmas lights (because of course it is), and it even has an experimental antenna that should be able to track the keys if you can boost its signal strength high enough; there's a little meter that indicates the antenna's signal strength by displaying 0-50 stars.

Your instincts tell you that in order to save Christmas, you'll need to get all fifty stars by December 25th.

Collect stars by solving puzzles. Two puzzles will be made available on each day in the Advent calendar; the second puzzle is unlocked when you complete the first. Each puzzle grants one star. Good luck!

As the submarine drops below the surface of the ocean, it automatically performs a sonar sweep of the nearby sea floor. On a small screen, the sonar sweep report (your puzzle input) appears: each line is a measurement of the sea floor depth as the sweep looks further and further away from the submarine.

For example, suppose you had the following report:

```
199
200
208
210
200
207
240
269
260
263
```

This report indicates that, scanning outward from the submarine, the sonar sweep found depths of 199, 200, 208, 210, and so on.

The first order of business is to figure out how quickly the depth increases, just so you know what you're dealing with - you never know if the keys will get carried into deeper water by an ocean current or a fish or something.

To do this, count the number of times a depth measurement increases from the previous measurement. (There is no measurement before the first measurement.) In the example above, the changes are as follows:

```
199 (N/A - no previous measurement)
200 (increased)
208 (increased)
210 (increased)
200 (decreased)
207 (increased)
240 (increased)
269 (increased)
260 (decreased)
263 (increased)
```

In this example, there are 7 measurements that are larger than the previous measurement.

How many measurements are larger than the previous measurement?

### Planning 

Goal: count the number of times a measurement increases from the previous measurement

Thoughts:
- A variable to represent the noun of "count of measurement increases"
- A comparison that answers the question if the measurement increased or not
- We'll run our algorithm on all but the first value, so we don't get an index out of range error.
- I'll likely need to convert the raw input to make sure it's not a string.
- Use the example input + output as a unit test

In [1]:
def count_measurement_increases(data):
    """
    Takes in a list of numbers and returns the number of times a measurement increases in value
    """
    
    # Count starts at zero
    count = 0
    
    # I'm using enumerate in order to access the index number of the list. 
    # The underscore variable is a convention for unused variables
    # I choose to use a for instead of a while loop b/c my input is finite and I won't run the risk of an infinite loop.
    for i, _ in enumerate(data):
        
        # Only run this algorithm if we're not on the very first element
        if i > 0:
            last_measurement = data[i - 1]
            current_measurement = data[i]

            if current_measurement > last_measurement:
                count += 1
                
    return count

In [2]:
example_data = [199, 200, 208, 210, 200, 207, 240, 269, 260, 263]
assert count_measurement_increases(example_data) == 7
print("Solution appears sufficient")

Solution appears sufficient


In [3]:
# Let's check with the puzzle input
# Aquire the data
filename = 'input1.txt'
with open(filename) as f:
    data = f.readlines()
    
# Check the data
data[0:10]

['196\n',
 '197\n',
 '176\n',
 '182\n',
 '179\n',
 '177\n',
 '171\n',
 '172\n',
 '170\n',
 '147\n']

In [4]:
# Wrangle the data
# Looks like we need to remove new line characters and convert the data type
data = [int(n.strip()) for n in data]

In [5]:
# Double check the data
data[0:10]

[196, 197, 176, 182, 179, 177, 171, 172, 170, 147]

In [6]:
# Puzzle Answer
count_measurement_increases(data)

1228

## Part 2

--- Part Two ---
Considering every single measurement isn't as useful as you expected: there's just too much noise in the data.

Instead, consider sums of a three-measurement sliding window. Again considering the above example:

```
199  A      
200  A B    
208  A B C  
210    B C D
200  E   C D
207  E F   D
240  E F G  
269    F G H
260      G H
263        H
```

Start by comparing the first and second three-measurement windows. The measurements in the first window are marked A (199, 200, 208); their sum is 199 + 200 + 208 = 607. The second window is marked B (200, 208, 210); its sum is 618. The sum of measurements in the second window is larger than the sum of the first, so this first comparison increased.

Your goal now is to count the number of times the sum of measurements in this sliding window increases from the previous sum. So, compare A with B, then compare B with C, then C with D, and so on. Stop when there aren't enough measurements left to create a new three-measurement sum.

In the above example, the sum of each three-measurement window is as follows:

```
A: 607 (N/A - no previous sum)
B: 618 (increased)
C: 618 (no change)
D: 617 (decreased)
E: 647 (increased)
F: 716 (increased)
G: 769 (increased)
H: 792 (increased)
```
In this example, there are 5 sums that are larger than the previous sum.

Consider sums of a three-measurement sliding window. How many sums are larger than the previous sum?

### Planning

- The goal is to count the number of times the sum of a window is greater than the last window (of 3 measurements)
- I might want a piece of state to hold each window. Up to 3 windows at a time. But there's often better options than adding extra state variables...
- Reshaping might be helpful, here. Consider how reshaping simplifies the entire affair. 
    ```
    [
        [199, 200, 208],
        [200, 208, 210],
        [208, 210, 200],
        ...
    ]
    ```
- My first thought is an array chunk or split, but I don't know that those built in functions move a window with some overlapping indexes.
- If reshaping costs too much time, then perhaps I only need two arrays in memory: `this_window` and `last_window` where they are each 3 element lists/arrays that I'll sum and then append the sums into a flat list. 
- Once I have a flat list of sums, I can rely on the previously defined `count_measurement_increases` to count the number of measurement (window) increases.
- `np.lib.stride_tricks.sliding_window_view(data, 3)` in numpy version >= 1.20 solves this exact sliding window view problem.
    - From the purist/practioner's perspective, it would be interesting and valuable to implement this sliding window view.
    - From the firm's view, it's not worth employee time to reimplement solutions available in proven, trusted libraries.
    - The A product owner or project manager's perspective may be closer in alignment with the firm's. Software should be upgraded and since dev time is expensive and we don't want to maintain someone's implementation, let's use numpy.
    - From the Dev and Ops team perspective, they might not be able to upgrade a dependency that may need to update other dependencies. Updates and dependencies can create a house of cards/hydra problem. But it's challenging to say "Let's freeze all of our dependencies" unless it's already been an expensive problem

In [7]:
import numpy as np

In [8]:
def count_window_increases(data):
    data = np.lib.stride_tricks.sliding_window_view(data, 3)
    sums = data.sum(axis=1)
    increases = count_measurement_increases(sums)
    return increases

In [9]:
# Use example data from above to check a unit test
example_data = [199, 200, 208, 210, 200, 207, 240, 269, 260, 263]
assert count_window_increases(example_data) == 5
print("Solution sufficient on example data")

Solution sufficient on example data


In [10]:
count_window_increases(data)

1257

# Other Approaches
- numpy/pandas for the first question
    - Using pandas [.diff](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.diff.html)
- adding visuals might be neat. 


In [12]:
import numpy as np
import pandas as pd

In [21]:
def pandas_part1(data):
    numbers = pd.Series(data)
    differences = numbers.diff()
    positive_differences = (differences > 0)
    return positive_differences.sum()

In [23]:
assert pandas_part1(data) == 1228
print("Satisfies the spec")

Satisfies the spec
